In [6]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
from object_detection.utils import model_util
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import label_map_util
from object_detection.utils import object_detection_eval
from object_detection.utils import dataset_util
import cv2
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

# Paths to the dataset
train_folder = 'C:/Users/naren/Downloads/Compressed/Tuna Loin Grade Detection.v1i.tensorflow/train'
train_csv = os.path.join(train_folder, '_annotations.csv')
val_folder = 'C:/Users/naren/Downloads/Compressed/Tuna Loin Grade Detection.v1i.tensorflow/valid'
val_csv = os.path.join(val_folder, '_annotations.csv')
test_folder = 'C:/Users/naren/Downloads/Compressed/Tuna Loin Grade Detection.v1i.tensorflow/test'
test_csv = os.path.join(test_folder, '_annotations.csv')

# Paths to TensorFlow model and config files for MobileNet SSDv2
pipeline_config_path = 'ssd_mobilenet_v2_coco.config'  # path to the config file
model_dir = 'ssd_mobilenet_v2_coco'  # path to store model

# Set up the pipeline configuration for SSD MobileNetv2
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()

# Load and parse the pipeline config
with tf.io.gfile.GFile(pipeline_config_path, 'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

# Modify the pipeline config (adjust paths, batch size, learning rate, etc.)
pipeline_config.model.ssd.num_classes = 3
pipeline_config.train_config.batch_size = 8
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(model_dir, 'checkpoint', 'model.ckpt')

# Apply the configuration changes
config_util.create_configs_from_pipeline_proto(pipeline_config, model_dir)

# Training the model (this requires a custom training script to run the pipeline)
# Use TensorFlow's Object Detection API for training
# The actual training script can be executed by running a script like:
# python model_main_tf2.py --pipeline_config_path=ssd_mobilenet_v2_coco.config --model_dir=model_dir --alsologtostderr

# After training, you can evaluate the model and visualize results

# Load trained model and test it
model = tf.saved_model.load(model_dir)

# Function to run the model on a given image and display results
def detect_objects(image_path):
    image_np = np.array(cv2.imread(image_path))
    input_tensor = tf.convert_to_tensor(image_np)
    input_tensor = input_tensor[tf.newaxis,...]

    # Run the detection
    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

    # Handle output results
    output_dict = {key:value.numpy() for key,value in output_dict.items()}
    boxes = output_dict['detection_boxes']
    classes = output_dict['detection_classes']
    scores = output_dict['detection_scores']
    
    return boxes, classes, scores

# Visualize and display detections
def display_detection(image_path, boxes, classes, scores):
    image_np = cv2.imread(image_path)
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        boxes,
        classes.astype(np.int32),
        scores,
        label_map_util.create_category_index_from_labelmap('label_map.pbtxt'),
        instance_masks=None,
        use_normalized_coordinates=True,
        line_thickness=8)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB))
    plt.show()

# Test detection with an image
image_path = 'path_to_sample_image.jpg'
boxes, classes, scores = detect_objects(image_path)
display_detection(image_path, boxes, classes, scores)


ImportError: cannot import name 'model_util' from 'object_detection.utils' (C:\Users\naren\AppData\Roaming\Python\Python311\site-packages\object_detection\utils\__init__.py)